In [ ]:
from selenium import webdriver
from urllib.parse import quote_plus
from urllib.request import urlopen
import os
from selenium.webdriver.common.by import By
import time

### 1. 페이지 스크롤 함수

In [ ]:
def scrollToEnd(driver):
    prev_height = driver.execute_script('return document.body.scrollHeight')
    print(f'prev_height: {prev_height}')
    
    while True:
        time.sleep(1) #네이버는 sleep없이 이동할 경우 무한로딩에 걸린다.
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(3)
        
        cur_height = driver.execute_script('return document.body.scrollHeight')
        print(f'cur_height: {cur_height}')
        if cur_height == prev_height:
            print('높이가 같아짐')
            break
        prev_height = cur_height
    
    # 페이지를 모두 로딩한 후에는 최상단으로 다시 올라가기
    driver.execute_script('window.scrollTo(0, 0)') 

### 2. 네이버 이미지 페이지에서 이미지 크롤링

In [ ]:
lst = []

for i in ['퍼그 이미지', '퍼그 미용']:
    url = f'https://search.naver.com/search.naver?ssc=tab.image.all&where=image&sm=tab_jum&query={i}'

    # chrome 브라우저 열기
    browser = webdriver.Chrome()
    browser.implicitly_wait(3)  # 브라우저를 오픈할 때 시간간격 제공
    browser.get(url)

    # 이미지 긁어오기
    while True:
        scrollToEnd(browser)
        images = browser.find_elements(By.CLASS_NAME, "_fe_image_tab_content_thumbnail_image")
        lst += images
        # 두 검색 항목을 모두 합해서 4000장을 크롤링하도록 조건문 수행
        if i == '퍼그 이미지':
            if len(lst) > 2000:
                break
        else:
            if len(lst) > 4000:
                break

### 3. 수집한 이미지 파일 경로를 실제 이미지로 변환 수행

In [ ]:
index = 0

for image in list(set(lst)):
    html_code = image.get_attribute('src')
    save_path = 'D:\\ai_phiso\dog\\fug\\'
    t = urlopen(html_code).read()
    file = open(os.path.join(save_path, "fug"+ str(index + 1) + ".jpg"), "wb")
    file.write(t)
    print("img save " + save_path + "fug" + str(index + 1) + ".jpg") 
    index += 1

### 4. 수집한 이미지에서 강아지만 검출 수행

In [ ]:
from ultralytics import YOLO
import cv2

for dog_name in os.listdir('D:\\ai_phiso\dog')[2:]:
    print(dog_name)
    
    model = YOLO('yolov8s.pt')

    folder_name = f'D:\\ai_phiso\dog\\{dog_name}' # 크롭할 이미지가 있는 폴더

    file_num = 1

    for file_name in os.listdir(f'{folder_name}'):
        img_name = f'{folder_name}\\{file_name}'
        img = cv2.imread(img_name)
        img = cv2.resize(img,(640,640))
        results = model.predict(img)

        try:
            a, b, c, d = [int(i) for i in (results[0].boxes.xyxy[0])]
            
            x1, y1 = a, b
            x2, y2 = c, d

            x1, x2 = min(a, c), max(a, c)
            y1, y2 = min(b, d), max(b, d)

            cropped_img = img[y1+2:y2-2, x1+2:x2-2] # 이미지 내 객체만 추출
            cropped_img_path = f'D:\\ai_phiso\\data\\{dog_name}_{file_num}.jpg' # 크롭한 이미지가 저장될 폴더 + 이미지 이름

            cv2.imwrite(cropped_img_path, cropped_img) # 크롭한 이미지 저장
            file_num += 1
        except:
            continue